In [1]:
import os
import sys
import boto3
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from pathlib import Path
from dotenv import load_dotenv

# defining functions module path
module_path = str(Path.cwd().parents[0] / "src")
if module_path not in sys.path:
    sys.path.append(module_path)

from functions import connect_to_redshift

In [2]:
# removing warning messages
warnings.filterwarnings("ignore")

In [3]:
# Loading my .env file
load_dotenv()

True

In [4]:
# defining redshift credentials
dbname = os.getenv("dbname")
host = os.getenv("host")
port = os.getenv("port")
user = os.getenv("user")
password = os.getenv("password")

In [5]:
# establishing redshift connection
rs_connection = connect_to_redshift(dbname, host, port, user, password)

connection to redshift made


In [6]:
query = """select *
           from bootcamp.online_transactions_cleaned
        """

In [7]:
ot_cleaned = pd.read_sql(query, rs_connection)

In [8]:
ot = ot_cleaned.copy()

In [9]:
ot_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 399841 entries, 0 to 399840
Data columns (total 9 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   invoice            399841 non-null  object        
 1   stock_code         399841 non-null  object        
 2   description        399841 non-null  object        
 3   price              399841 non-null  float64       
 4   quantity           399841 non-null  int64         
 5   total_order_value  399841 non-null  float64       
 6   invoice_date       399841 non-null  datetime64[ns]
 7   customer_id        399841 non-null  object        
 8   country            399841 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(5)
memory usage: 27.5+ MB


In [10]:
ot_cleaned.describe()

,price,quantity,total_order_value
count,399841.000000,399841.000000,399841.000000
mean,2.952914,12.224359,20.716904
std,7.317593,250.788860,425.471765
min,0.000000,-80995.000000,-168469.600000
25%,1.250000,2.000000,4.250000
50%,1.950000,5.000000,11.580000
75%,3.750000,12.000000,19.500000
max,1599.260000,80995.000000,168469.600000


In [11]:
no_price_stock = ot_cleaned[ot_cleaned.price <= 0]['stock_code'].unique()
print(f"There are {ot_cleaned[ot_cleaned.price <= 0].shape[0]} transactions with price equal to zero")
ot_cleaned[ot_cleaned['stock_code'].isin(no_price_stock)]

There are 34 transactions with price equal to zero


,invoice,stock_code,description,price,quantity,total_order_value,invoice_date,customer_id,country
60,C536548,22580,ADVENT CALENDAR GINGHAM SACK,5.95,-4,-23.80,2010-12-01 14:33:00,u12472,Germany
98,536464,22619,SET OF 6 SOLDIER SKITTLES,3.75,1,3.75,2010-12-01 12:23:00,u17968,United Kingdom
115,536667,22423,REGENCY CAKESTAND 3 TIER,12.75,4,51.00,2010-12-02 12:09:00,u1526,United Kingdom
144,536947,22423,REGENCY CAKESTAND 3 TIER,12.75,3,38.25,2010-12-03 12:29:00,u13013,United Kingdom
251,537201,22636,CHILDS BREAKFAST SET CIRCUS PARADE,8.50,4,34.00,2010-12-05 14:19:00,u12472,Germany
...,...,...,...,...,...,...,...,...,...
399335,581230,22090,PAPER BUNTING RETROSPOT,2.95,12,35.40,2011-12-08 10:24:00,u14291,United Kingdom
399369,579149,22619,SET OF 6 SOLDIER SKITTLES,3.75,4,15.00,2011-11-28 13:21:00,u13844,United Kingdom
399384,579165,22619,SET OF 6 SOLDIER SKITTLES,3.75,4,15.00,2011-11-28 14:01:00,u13389,United Kingdom
399579,580536,22619,SET OF 6 SOLDIER SKITTLES,3.75,4,15.00,2011-12-04 16:07:00,u1249,France


### Even though there are some transactions with stocks with price equal to zero, they having been also purchases with the same stocks having price bigger than 0. What happened there?

In [12]:
no_price = ot_cleaned[ot_cleaned.price <= 0]

In [13]:
print(f"There are {(ot_cleaned.quantity < 0).sum()} entries with negative quantity")

There are 8507 entries with negative quantity


In [14]:
ot_cleaned['invoice'][ot_cleaned.quantity < 0]

60        C536548
139       C536820
150       C536548
205       C536850
252       C536807
           ...   
399631    C580968
399708    C581464
399729    C581464
399753    C580711
399834    C581462
Name: invoice, Length: 8507, dtype: object

### The total order value is equal to price * quantity, therefore the negavite values in this column

In [15]:
# Checking is all the invoice with negative quantity start with a letter C 
ot_cleaned['invoice'][ot_cleaned.quantity < 0].str.startswith('C').sum()

8507

### I will consider all the entries with negative quantity as returns. They have a C in the invoice code, which I will assume refers to a CANCELLED transaction

In [16]:
ot_cancelled = ot_cleaned[ot_cleaned.invoice.str.startswith('C')]

### I will clean my ot_cleaned by removing the transactions with negative quantity and the transactions with price equal to zero

In [17]:
# Dropping cancelled orders
ot_cleaned.drop(ot_cancelled.index, axis=0, inplace=True)

In [18]:
# Dropping transaction with no price
ot_cleaned.drop(no_price.index, axis=0, inplace=True)

In [19]:
ot_cleaned.describe(include=object)

,invoice,stock_code,description,customer_id,country
count,391300,391300,391300,391300,391300
unique,18405,3662,3627,4334,37
top,576339,85123A,CREAM HANGING HEART T-LIGHT HOLDER,u17841,United Kingdom
freq,542,2023,2023,7667,348919


### There are transactions purchased in 37 different countries. The number of unique descriptions is not compatible with the number of unique stock codes. Why?

In [20]:
# Getting the data grouped by description and unique stock_code
descr_group = ot_cleaned.groupby('description').agg({'stock_code': 'unique'})


# Filtering just the descriptions with more than one stock_code
descr_group[ot_cleaned.groupby('description').agg({'stock_code': 'nunique'}).values!=1]

,stock_code
description,
COLOURING PENCILS BROWN TUBE,"[10135, 10133]"
COLUMBIAN CANDLE RECTANGLE,"[72133, 72131]"
COLUMBIAN CANDLE ROUND,"[72130, 72128]"
EAU DE NILE JEWELLED PHOTOFRAME,"[85023B, 85024B]"
FRENCH FLORAL CUSHION COVER,"[20840, 20837]"
FRENCH LATTICE CUSHION COVER,"[20838, 20835]"
FROSTED WHITE BASE,"[79403, 79406]"
"METAL SIGN,CUPCAKE SINGLE HOOK","[82613B, 82613C, 82613A]"
PINK FAIRY CAKE CUSHION COVER,"[47586A, 47585A]"


### It is interesting to see that there are some stock codes with 'Unknown' description. Concerning the descriptions with more than one stock code, I will infer that they are the same product on different presentations (i.e size)

In [21]:
# Filtering the data to get the number of stock codes with 'Unknown' description
stock_code_unknown = ot_cleaned[ot_cleaned.description == 'Unknown']['stock_code'].unique()
print(f"There are {stock_code_unknown.shape[0]} stock codes with unknown description")

There are 18 stock codes with unknown description


In [22]:
# Checking if the stock codes with unknown description were sold in all countries
ot_cleaned[ot_cleaned['stock_code'].isin(stock_code_unknown)].country.unique()

array(['United Kingdom', 'Belgium', 'Switzerland', 'France',
       'Unspecified', 'EIRE', 'Poland', 'Germany', 'Spain', 'Norway',
       'Singapore', 'Sweden', 'Austria', 'Canada', 'Italy'], dtype=object)

### The Unknown description purchases were made across 15 of the 37 countries. The reason why they are Unknown is still unclear

### Let's explore each column

In [23]:
print(f"The data contains information about 391334 ({ot_cleaned.invoice.nunique()} invoices in total) online purchases made from {ot_cleaned.invoice_date.min()} to {ot_cleaned.invoice_date.max()}. {ot_cancelled.shape[0]} are cancelled orders")

The data contains information about 391334 (18405 invoices in total) online purchases made from 2010-12-01 08:26:00 to 2011-12-09 12:50:00. 8507 are cancelled orders


In [24]:
print(f"A total of {ot_cleaned.stock_code.nunique()} stock codes were sold from which {ot_cancelled.stock_code.nunique()} stocks were return at least once")

A total of 3662 stock codes were sold from which 1916 stocks were return at least once


In [25]:
inv_country = ot_cleaned.groupby('country').count()['invoice'].sort_values(ascending=False)

In [26]:
print(f"The invoices were purchased in 37 countries being {inv_country.index[0]} the country with the most purchases ({inv_country[0]}) and {inv_country.index[-1]} with the least ({inv_country[-1]})")

The invoices were purchased in 37 countries being United Kingdom the country with the most purchases (348919) and Saudi Arabia with the least (9)


In [27]:
print(f"In total, there are {ot_cleaned.customer_id.nunique()} costumers")

In total, there are 4334 costumers


### In the table below are shown the top 10 stock codes sold, their description, amount sold, their price and in how many country they had been sold

In [37]:
# Let's get top 5 stock codes on the table 
# defining aggregation functions to show on the table
agg_dict = {'description': 'first', 'quantity': 'sum', 'price': 'first', 'country': 'nunique'}
top10 = ot_cleaned.groupby("stock_code").agg(agg_dict).sort_values(by='quantity', ascending=False)[:10]
top10

,description,quantity,price,country
stock_code,,,,
23843,"PAPER CRAFT , LITTLE BIRDIE",80995,2.08,1
23166,MEDIUM CERAMIC TOP STORAGE JAR,77916,1.25,10
84077,WORLD WAR 2 GLIDERS ASSTD DESIGNS,54319,0.29,12
22197,POPCORN HOLDER,49160,0.85,14
85099B,JUMBO BAG RED RETROSPOT,46078,1.95,20
85123A,CREAM HANGING HEART T-LIGHT HOLDER,36763,2.95,16
84879,ASSORTED COLOUR BIRD ORNAMENT,35263,1.69,16
21212,PACK OF 72 RETROSPOT CAKE CASES,33670,0.42,20
23084,RABBIT NIGHT LIGHT,27153,2.08,19


In [38]:
# What are the countries with the highest purchase
ot.groupby('country').sum().sort_values(by='total_order_value', ascending=False)[:10]

,price,quantity,total_order_value
country,,,
United Kingdom,1045296.903,3992342,6815375.093
Netherlands,5691.930,200510,283479.540
EIRE,29904.890,136189,251557.470
Germany,25266.750,116235,200619.660
France,21988.260,109019,181571.540
Australia,3435.270,83643,136922.500
Switzerland,5326.600,29681,51859.400
Spain,8042.880,26611,51746.650
Belgium,6037.130,22880,36662.960


### On the other hand, we can also inspect the top X of any contry. To see the tables please remove the comments on the two cells below

In [30]:
# country = input("Type a country from the previous list") 
# top = int(input("How many items do you want to see in the top list"))

In [31]:
# grouping data by country
# country_group = ot_cleaned.groupby("country")

# country_group.get_group(country).groupby("stock_code").agg(agg_dict).sort_values(by='quantity', ascending=False)[:top]

### Let's explore now the data with returns

In [39]:
ot_cancelled.groupby("stock_code").agg(agg_dict).sort_values(by='quantity')[:10]

,description,quantity,price,country
stock_code,,,,
23843,"PAPER CRAFT , LITTLE BIRDIE",-80995,2.08,1
23166,MEDIUM CERAMIC TOP STORAGE JAR,-74494,1.04,5
84347,ROTATING SILVER ANGELS T-LIGHT HLDR,-9367,0.03,2
21108,FAIRY CAKE FLANNEL ASSORTED COLOUR,-3150,2.10,1
85123A,CREAM HANGING HEART T-LIGHT HOLDER,-2578,2.55,3
21175,GIN AND TONIC DIET METAL SIGN,-2030,1.85,2
22920,HERB MARKER BASIL,-1527,0.55,1
22273,FELTCRAFT DOLL MOLLY,-1447,2.95,2
47566B,TEA TIME PARTY BUNTING,-1424,4.25,1


In [40]:
# Checking how many stock codes from the top10 best sellers are in the returned list
canc_grouped = ot_cancelled.groupby('stock_code').sum()['quantity'].sort_values()[:10]
top10.index.isin(canc_grouped.index).sum()

3

### Inspecting the returned items, I realized that in the top 10 most returned items are some of the best selling items. I will create a new version of the dataset by removing the transactions that were return from the cleaned data

In [34]:
ot_cancelled.groupby("customer_id").agg(agg_dict)

,description,quantity,price,country
customer_id,,,,
u12346,MEDIUM CERAMIC TOP STORAGE JAR,-74215,1.04,1
u12352,PINK DOG BOWL,-63,2.95,1
u12359,RECIPE BOX PANTRY YELLOW DESIGN,-10,2.95,1
u12362,SPACEBOY LUNCH BOX,-17,1.95,1
u12375,JUMBO BAG BAROQUE BLACK WHITE,-1,2.08,1
...,...,...,...,...
u18272,PINK POT PLANT CANDLE,-6,2.95,1
u18274,REGENCY CAKESTAND 3 TIER,-88,12.75,1
u18276,RUSTIC MIRROR WITH LACE HEART,-2,6.25,1


### 1534 customers returned a product at least once

In [35]:
ot_cancelled.groupby("stock_code").agg(agg_dict)

,description,quantity,price,country
stock_code,,,,
10133,COLOURING PENCILS BROWN TUBE,-10,0.42,1
10135,COLOURING PENCILS BROWN TUBE,-1,1.25,1
11001,ASSTD DESIGN RACING CAR PEN,-185,1.69,1
15034,PAPER POCKET TRAVELING FAN,-1385,0.14,1
15036,ASSORTED COLOURS SILK FAN,-744,0.65,1
...,...,...,...,...
90208,PAIR OF PINK FLOWER CLUSTER SLIDE,-6,2.55,1
90209A,PURPLE ENAMEL+GLASS HAIR COMB,-6,2.10,1
90209C,PINK ENAMEL+GLASS HAIR COMB,-3,2.10,1


### 1916 have been returned at least once